In [ ]:
%pip install msfabricpysdkcore --quiet

In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from msfabricpysdkcore import FabricClientCore
try:
    import sempy.fabric as fabric
    local = False
except:
    local = True
    from azure.identity import DefaultAzureCredential
import pyodbc
import struct
import pandas as pd
from time import sleep

In [ ]:
sql_database_name = "metadatamirroring"
mirrored_db_name = "snowflakeviewmirror"

sql_database_workspace_id = None#"141103ea-1119-4e51-9ee2-aa4ffba79577"
mirrored_db_workspace_id = None#"141103ea-1119-4e51-9ee2-aa4ffba79577"

# If you are running this notebook locally, please set the workspace ID to the one you are using in your Fabric workspace.
# If you are running this notebook in Fabric, please leave the workspace ID as None.
# The code will automatically get the workspace ID of the workspace you are using for this notebook.

if sql_database_workspace_id is None:
    sql_database_workspace_id = fabric.get_workspace_id()
if mirrored_db_workspace_id is None:
    mirrored_db_workspace_id = fabric.get_workspace_id()


In [ ]:
fcc = FabricClientCore()

In [ ]:
sql_database_workspace_id = sql_database_workspace_id if sql_database_workspace_id else fabric.get_workspace_id()
sql_dbs = fcc.list_sql_databases(workspace_id=sql_database_workspace_id)
sql_db = [sql_db for sql_db in sql_dbs if sql_db.display_name == sql_database_name]

if len(sql_db) < 1:
    sql_db = fcc.create_sql_database(workspace_id=sql_database_workspace_id, display_name = sql_database_name)
else:
    sql_db = sql_db[0]
try:
    server = sql_db.properties["serverFqdn"][:-5]
    databasename = sql_db.properties["databaseName"]
    database = "{" + f"{databasename}" +"}"
except:
    sleep(10)
    sql_database_workspace_id = sql_database_workspace_id if sql_database_workspace_id else fabric.get_workspace_id()
    sql_dbs = fcc.list_sql_databases(workspace_id=sql_database_workspace_id)
    sql_db = [sql_db for sql_db in sql_dbs if sql_db.display_name == sql_database_name]
    sql_db = sql_db[0]

In [ ]:
# Get Azure token using DefaultAzureCredential
def run_query(server, database, query, query_type="SELECT"):
    if local:
        # Get Azure token using DefaultAzureCredential
        credential = DefaultAzureCredential()
        token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
    else:
        token_bytes = notebookutils.credentials.getToken("https://database.windows.net/.default").encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
    SQL_COPT_SS_ACCESS_TOKEN = 1256  # This connection option is defined by microsoft in msodbcsql.h

    # Connection parameters
    connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={server};Database={database};"

    # Connect with Entra ID (Azure AD) token
    conn = pyodbc.connect(connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})
    cursor = conn.cursor()

    # Test the connection
    cursor.execute(query)
    if query_type=="SELECT":
        rows = cursor.fetchall()

        column_names = [column[0] for column in cursor.description]
        return_value = pd.DataFrame.from_records(rows, columns=column_names)
    else:
        cursor.execute("COMMIT;")
        return_value = "success"

    # Close the connection
    cursor.close()
    conn.close()
    return return_value
    


In [ ]:
query = """
CREATE TABLE [dbo].[Metadata](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[workspace_id] [nvarchar](255) NOT NULL,
	[mirrored_db_id] [nvarchar](255) NOT NULL,
	[snowflake_db] [nvarchar](255) NOT NULL,
	[snowflake_schema] [nvarchar](255) NOT NULL,
	[view_name] [nvarchar](255) NOT NULL,
	[identifiers] [nvarchar](max) NULL,
	[streamable] [nvarchar](255) NOT NULL,
	[status] [nvarchar](30) NOT NULL,
	[keyvault] [nvarchar](30) NOT NULL)
	;
"""
run_query(server,database,query,query_type="CREATE")


In [ ]:
query = """
ALTER TABLE [dbo].[Metadata] ADD  DEFAULT ('unknown') FOR [streamable]"""
run_query(server,database,query,query_type="CREATE")

In [ ]:
query = """
ALTER TABLE [dbo].[Metadata] ADD  DEFAULT ('not initialized') FOR [status]
"""
run_query(server,database,query,query_type="CREATE")

In [ ]:
query = """
CREATE TABLE [dbo].[mirroringruns](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[mirrored_view_id] [nvarchar](255) NOT NULL,
    [inserted_rows] [int] NOT NULL,
    [deleted_rows] [int] NOT NULL,
    [timestamp] [datetime] NOT NULL,
    [status] [nvarchar](30) NOT NULL
)"""
run_query(server,database,query,query_type="CREATE")

In [ ]:
query = """ CREATE VIEW summary AS 
SELECT 
      m.[workspace_id],
      m.[mirrored_db_id],
      m.[snowflake_db],
      m.[snowflake_schema],
      m.[view_name],
      SUM(mr.inserted_rows) as sum_inserted_rows,
      SUM(mr.deleted_rows) as sum_deleted_rows,
      MAX(mr.timestamp) as last_update
      FROM [dbo].[Metadata] m
LEFT OUTER JOIN [dbo].[mirroringruns] mr ON m.[ID] = mr.[mirrored_view_id]
GROUP BY
m.[workspace_id],
m.[mirrored_db_id],
m.[snowflake_db],
m.[snowflake_schema],
m.[view_name] 
"""
run_query(server,database,query,query_type="CREATE")

In [ ]:
if mirrored_db_name:
    mirrored_db_workspace_id = mirrored_db_workspace_id if mirrored_db_workspace_id else fabric.get_workspace_id()
    definition = {'parts': [{'path': 'mirroring.json',
                                'payload': 'ew0KICAicHJvcGVydGllcyI6IHsNCiAgICAic291cmNlIjogew0KICAgICAgInR5cGUiOiAiR2VuZXJpY01pcnJvciIsDQogICAgICAidHlwZVByb3BlcnRpZXMiOiBudWxsDQogICAgfSwNCiAgICAidGFyZ2V0Ijogew0KICAgICAgInR5cGUiOiAiTW91bnRlZFJlbGF0aW9uYWxEYXRhYmFzZSIsDQogICAgICAidHlwZVByb3BlcnRpZXMiOiB7DQogICAgICAgICJmb3JtYXQiOiAiRGVsdGEiLA0KICAgICAgICAiZGVmYXVsdFNjaGVtYSI6ICJkYm8iDQogICAgICB9DQogICAgfQ0KICB9DQp9',
                                'payloadType': 'InlineBase64'},
                            {'path': '.platform',
                                'payload': 'ewogICIkc2NoZW1hIjogImh0dHBzOi8vZGV2ZWxvcGVyLm1pY3Jvc29mdC5jb20vanNvbi1zY2hlbWFzL2ZhYnJpYy9naXRJbnRlZ3JhdGlvbi9wbGF0Zm9ybVByb3BlcnRpZXMvMi4wLjAvc2NoZW1hLmpzb24iLAogICJtZXRhZGF0YSI6IHsKICAgICJ0eXBlIjogIk1pcnJvcmVkRGF0YWJhc2UiLAogICAgImRpc3BsYXlOYW1lIjogIk1pcnJvcmVkRGF0YWJhc2VfMSIKICB9LAogICJjb25maWciOiB7CiAgICAidmVyc2lvbiI6ICIyLjAiLAogICAgImxvZ2ljYWxJZCI6ICIwMDAwMDAwMC0wMDAwLTAwMDAtMDAwMC0wMDAwMDAwMDAwMDAiCiAgfQp9',
                                'payloadType': 'InlineBase64'}]}

    mirrored_db = fcc.create_mirrored_database(mirrored_db_workspace_id, mirrored_db_name, definition=definition)
mirrored_db.id